In [1]:
import pandas as pd
from math import sqrt
import datetime
import numpy as np
import re

In [2]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/animesDF.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores.csv')

In [9]:
print(usersDF.shape, animesDF.shape, scoresDF.shape)

(108711, 17) (2035, 33) (3434652, 4)


In [4]:
anime_ids = np.sort(animesDF.anime_id.unique())
len(anime_ids)

2035

In [5]:
top_10000_users = list(usersDF.sort_values(by=['user_completed'], ascending=False).head(10000)['username'])

In [6]:
#scoresDF = scoresDF.sample(n=10000, random_state=15)
#scoresDF.to_csv('../../../Downloads/myanimelist/scores_sample.csv', index=False)

In [7]:
scoresDF = scoresDF[scoresDF['username'].isin(top_10000_users)]

In [8]:
scoresDF = scoresDF[scoresDF['anime_id'].isin(anime_ids)]

In [8]:
animesDF = animesDF[(animesDF['duration_min']>=20) \
                    & (animesDF['aired_from_year']>=2000) \
                    & ~(animesDF.related.str.contains('Prequel')) \
                    & (animesDF['type'] != 'Special') \
                    & ~(animesDF.genre.str.contains('Hentai')) \
                    & ~(animesDF.genre.str.contains('Kids')) \
                    & ~(animesDF.genre.str.contains('Shoujo Ai')) \
                    & ~(animesDF.genre.str.contains('Shounen Ai')) \
                    & ~(animesDF.genre.str.contains('Yaoi')) \
                    & ~(animesDF.genre.str.contains('Yuri'))]
animesDF.to_csv('../../../Downloads/myanimelist/animesDF.csv', index=False)

In [10]:
animesDF.sort_values(by=['anime_id']).head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
1586,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0
1456,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,Tuesdays at Unknown,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0
1336,8,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '...",23.0,2004.0
864,15,Eyeshield 21,NaN,アイシールド21,Eyeshield21,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,145,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 43, 'type': 'manga'...","TV Tokyo, Nihon Ad Systems, TV Tokyo Music, Sh...",Sentai Filmworks,Studio Gallop,"Action, Sports, Comedy, Shounen","['#1: ""Breakthrough"" by Coming Century (eps 1-...","['#1: ""Be Free"" by Rikken&#039;z (eps 1-13)', ...",23.0,2005.0
208,16,Hachimitsu to Clover,Honey and Clover,ハチミツとクローバー,"HachiKuro, Honey & Clover",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,24,Finished Airing,...,Fridays at 00:35 (JST),"{'Adaptation': [{'mal_id': 1009, 'type': 'mang...","Genco, Fuji TV, Shueisha",Viz Media,J.C.Staff,"Comedy, Drama, Josei, Romance, Slice of Life","['""Dramatic"" by Yuki']","['#1: ""Waltz"" by Suneohair (eps 1-12, 24)', '#...",23.0,2005.0


In [11]:
scoresDF.head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [12]:
scoresDF[scoresDF.anime_id == 3514].max()

username     edinaa
anime_id       3514
my_score          8
my_status         4
dtype: object

In [13]:
anime_users = scoresDF.pivot(index='anime_id', columns='username', values='my_score')

In [14]:
anime_users.head()

username,--FallenAngel--,--kitade--,-Ackerman,-Alberto-,-Alians-,-Araragi-,-Azazel-,-BakaSama-,-Balmung-,-BlackStar,...,zombie_pegasus,zombie_wolf,zombisteva,zombro93,zoroUAE,zoropp,ztugrog,zwars,zwil,zzganz
anime_id,,,,,,,,,,,,,,,,,,,,,
5,NaN,NaN,8.0,NaN,NaN,9.0,9.0,NaN,8.0,8.0,...,NaN,NaN,7.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN
7,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,9.0,6.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
15,8.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,7.0,NaN,NaN,7.0,NaN,9.0
16,9.0,NaN,7.0,NaN,NaN,8.0,10.0,NaN,NaN,NaN,...,6.0,8.0,NaN,10.0,NaN,NaN,NaN,NaN,8.0,9.0


In [15]:
'''
gets all user scores
'''
def get_user_scores(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

get_user_scores(scoresDF, 'MistButterfly').head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [16]:
# rewrite all with new dataframe, speed increase expected
def sim_pearson(anime_users, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    both_scores = anime_users.loc[[anime1_id, anime2_id],:].dropna(axis='columns').to_numpy()
    scores_anime1 = both_scores[0]
    scores_anime2 = both_scores[1]
    
    shared_viewers = len(scores_anime1)
    
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = np.sum(scores_anime1)
    sum2 = np.sum(scores_anime2)
    
    # sum up the squares
    sum1Sq = np.sum(scores_anime1**2)
    sum2Sq = np.sum(scores_anime2**2)
    
    # sum up the products
    #pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    pSum = np.dot(scores_anime1, scores_anime2)
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return round(num/den, 10)

In [17]:
sim_pearson(anime_users, 5, 16)

0.327813189

In [18]:
'''
builds table of similarity of the shows
a[i][j] - similarity of the shows with ids: anime_ids[i] and anime_ids[j]
'''
def get_similarity_table(scores, anime_ids):
    similarity_table = np.load('similarity_table.npy')
    n = len(anime_ids)
    counter1 = 0
    counter2 = 0
    for anime1_id in anime_ids:
        counter2 = 0
        for anime2_id in anime_ids:
            if anime1_id != anime2_id:
                try:
                    similarity = sim_pearson(scores, anime1_id, anime2_id)
                except:
                    similarity = 0
                similarity_table[counter1][counter2] = similarity
                similarity_table[counter2][counter1] = similarity
                counter2 += 1
        counter1 += 1
        print('Processed {} of {} animes :: {}'.format(counter1, n, datetime.datetime.now()))
        np.save('similarity_table', similarity_table)
    return similarity_table

In [127]:
print(datetime.datetime.now())
get_similarity_table(anime_users, anime_ids)

2020-02-03 16:26:03.679100
Processed 1884 of 2084 animes :: 2020-02-03 16:26:11.623198
Processed 1885 of 2084 animes :: 2020-02-03 16:26:19.294286
Processed 1886 of 2084 animes :: 2020-02-03 16:26:26.150058
Processed 1887 of 2084 animes :: 2020-02-03 16:26:33.887477
Processed 1888 of 2084 animes :: 2020-02-03 16:26:42.103070
Processed 1889 of 2084 animes :: 2020-02-03 16:26:54.187567
Processed 1890 of 2084 animes :: 2020-02-03 16:27:04.361663
Processed 1891 of 2084 animes :: 2020-02-03 16:27:16.630238
Processed 1892 of 2084 animes :: 2020-02-03 16:27:26.798177
Processed 1893 of 2084 animes :: 2020-02-03 16:27:36.343623
Processed 1894 of 2084 animes :: 2020-02-03 16:27:47.309862
Processed 1895 of 2084 animes :: 2020-02-03 16:27:55.797878
Processed 1896 of 2084 animes :: 2020-02-03 16:28:05.680259
Processed 1897 of 2084 animes :: 2020-02-03 16:28:15.568679
Processed 1898 of 2084 animes :: 2020-02-03 16:28:27.119809
Processed 1899 of 2084 animes :: 2020-02-03 16:28:37.995801
Processed 190

Processed 2021 of 2084 animes :: 2020-02-03 16:44:45.264503
Processed 2022 of 2084 animes :: 2020-02-03 16:44:52.525904
Processed 2023 of 2084 animes :: 2020-02-03 16:44:59.651261
Processed 2024 of 2084 animes :: 2020-02-03 16:45:07.295327
Processed 2025 of 2084 animes :: 2020-02-03 16:45:14.278505
Processed 2026 of 2084 animes :: 2020-02-03 16:45:21.484366
Processed 2027 of 2084 animes :: 2020-02-03 16:45:28.663344
Processed 2028 of 2084 animes :: 2020-02-03 16:45:35.746309
Processed 2029 of 2084 animes :: 2020-02-03 16:45:43.114525
Processed 2030 of 2084 animes :: 2020-02-03 16:45:50.212885
Processed 2031 of 2084 animes :: 2020-02-03 16:45:57.456232
Processed 2032 of 2084 animes :: 2020-02-03 16:46:04.652750
Processed 2033 of 2084 animes :: 2020-02-03 16:46:12.152190
Processed 2034 of 2084 animes :: 2020-02-03 16:46:19.453424
Processed 2035 of 2084 animes :: 2020-02-03 16:46:26.872571
Processed 2036 of 2084 animes :: 2020-02-03 16:46:34.035996
Processed 2037 of 2084 animes :: 2020-02

array([[ 0.30246524,  0.30246524,  0.24109927, ...,  0.        ,
         0.23970106,  0.42921624],
       [ 0.30246524,  0.21368076,  0.21368076, ...,  0.        ,
         0.54772256,  0.47719134],
       [ 0.24109927,  0.21368076,  0.53160617, ...,  0.        ,
         1.        , -0.21821789],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.23970106,  0.54772256,  1.        , ...,  0.        ,
         0.69266029,  0.69266029],
       [ 0.42921624,  0.47719134, -0.21821789, ...,  0.        ,
         0.69266029,  0.        ]])

In [19]:
'''
loads user's scores from MAL xml file converted to csv
'''
def load_user_list(path):
    userDF = pd.read_csv(path)
    return userDF[['series_animedb_id', 'my_score']][userDF['my_score']!=0]

In [20]:
cmlimmDF = load_user_list('../../../Downloads/animelist_cmlimm.csv')

In [21]:
barbieDF = load_user_list('../../../Downloads/animelist_barbie.csv')

In [22]:
'''
find user's shows that exist in MAL dataset, as some shows aired after dataset was created
'''
def user_shows_in_db(userDF, animesDF):
    return userDF[userDF[['series_animedb_id']].isin(animesDF.anime_id.unique()).any(1)]

In [23]:
np.array(user_shows_in_db(cmlimmDF, animesDF))

array([[22147,     9],
       [ 2251,    10],
       [35965,     7],
       [28805,     7],
       [31964,     8],
       [33929,     8],
       [35459,     8],
       [16592,     6],
       [35849,     8],
       [ 6746,     8],
       [ 9314,     7],
       [33731,     5],
       [ 2890,     7],
       [18229,     7],
       [20583,     9],
       [25303,     7],
       [35806,     7],
       [27989,    10],
       [30415,     8],
       [10357,     9],
       [ 5680,     6],
       [ 9617,     6],
       [32686,     7],
       [18679,     9],
       [32281,     8],
       [31798,     4],
       [28851,     9],
       [30831,     7],
       [34712,     9],
       [33489,     9],
       [35677,     7],
       [15051,     7],
       [32526,     2],
       [34599,     8],
       [21603,     5],
       [10161,     8],
       [21557,     7],
       [31559,     6],
       [ 6675,     5],
       [ 5671,     7],
       [10884,     7],
       [29786,     6],
       [16498,     6],
       [ 61

In [24]:
'''
finds top matches based on shows similarities
'''
def top_matches(similarity_table, user_show_scores, anime_ids):
    result = np.empty((0,2), int)
    user_shows = user_show_scores[:, 0]
    user_scores = user_show_scores[:, 1]
    rows = np.where(np.isin(anime_ids, user_shows))[0]
    columns = np.setdiff1d(np.arange(len(anime_ids)), rows)
    matches_table = similarity_table[rows][:, columns]
    for i in range(len(columns)):
        result = np.append(result, [[int(anime_ids[columns[i]]), np.dot(matches_table[:, i], user_scores)/np.sum(matches_table[:, i])]], axis=0)
    return result[result[:,1].argsort()][::-1]

In [31]:
similarity_table = np.load('similarity_table.npy')

In [32]:
def show_names(matches, animesDF):
    data = {'anime_id': [], 'pred_score':[]}
    for show in matches:
        try:
            if show[1] <= 10:
                data['anime_id'].append(int(show[0]))
                data['pred_score'].append(show[1])
        except:
            pass
    return data

In [33]:
cmlimm_rec = show_names(top_matches(similarity_table, np.array(user_shows_in_db(cmlimmDF, animesDF)), anime_ids), animesDF)

/Users/cmlimm/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [34]:
cmlimm_recDF = pd.DataFrame(data=cmlimm_rec)
cmlimm_recDF.head()

,anime_id,pred_score
0,2382,8.602706
1,8476,8.309246
2,6974,8.277074
3,36023,8.164312
4,26,8.148885


In [36]:
animesDF.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,178,Ultra Maniac,Ultramaniac - Magical Girl,ウルトラマニアック,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Tuesdays at 22:00 (JST),"{'Adaptation': [{'mal_id': 956, 'type': 'manga...","WAO World, Studio Jack","Discotek Media, Geneon Entertainment USA",Production Reed,"Magic, Comedy, Romance, School, Shoujo","['""Kagami no Naka"" by can/goo']","['""Hitotsu=Unmei Kyoudoutai"" by can/goo']",24.0,2003.0
3,853,Ouran Koukou Host Club,Ouran High School Host Club,桜蘭高校ホスト部,"Ohran Koko Host Club, Ouran Koukou Hosutobu, O...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Wednesdays at 00:50 (JST),"{'Adaptation': [{'mal_id': 30, 'type': 'manga'...","VAP, Hakusensha, Nippon Television Network",Funimation,Bones,"Comedy, Harem, Romance, School, Shoujo","['""Sakura Kiss"" by Chieko Kawabe']","['""Shissou"" by Last Alliance']",23.0,2006.0
4,7054,Kaichou wa Maid-sama!,Maid Sama!,会長はメイド様!,Class President is a Maid!,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Fridays at 01:55 (JST),"{'Adaptation': [{'mal_id': 2921, 'type': 'mang...","Geneon Universal Entertainment, TBS, Hakusensh...",Sentai Filmworks,J.C.Staff,"Comedy, Romance, School, Shoujo","['""My Secret"" by Saaya Mizuno (水野佐彩)']","['#1: ""Yokan (予感)"" by heidi. (eps 1-15)', '#2:...",24.0,2010.0


In [60]:
resultDF = pd.merge(cmlimm_recDF, animesDF, on=['anime_id'])
resultDF[resultDF.pred_score > 7.5]

,anime_id,pred_score,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,2382,8.602706,Firestorm,NaN,FIRESTORM ファイアーストーム,NaN,https://myanimelist.cdn-dena.com/images/anime/11/6090.jpg,TV,Original,26,...,Sundays at 09:00 (JST),[],"Madhouse, TV Tokyo",Enoki Films,Trans Arts,"Action, Military, Sci-Fi, Adventure, Mecha","['""Stormy Love"" by the NaB&#039;s']","['#1: ""Kimi to Ita Chikyuu (君といた地球)"" by The NaB&#039;s (eps 1-13)', '#2: ""RE-SET"" by TAKUI (eps ...",25.0,2003.0
1,8476,8.309246,Otome Youkai Zakuro,Zakuro,おとめ妖怪 ざくろ,Girl Demon Zakuro,https://myanimelist.cdn-dena.com/images/anime/5/85430.jpg,TV,Manga,13,...,Tuesdays at 01:30 (JST),"{'Adaptation': [{'mal_id': 9084, 'type': 'manga', 'url': 'https://myanimelist.net/manga/9084/Oto...",Aniplex,"NIS America, Inc.",J.C.Staff,"Demons, Historical, Military, Romance, Seinen, Supernatural","['""Moon SIGNAL"" by Sphere']","['#1: ""Hatsukoi wa Zakuroiro (初戀は柘榴色)"" by Mai Nakahara with narration by Takahiro Sakurai (eps 1...",24.0,2010.0
2,6974,8.277074,Seikimatsu Occult Gakuin,Occult Academy,世紀末オカルト学院,"Zaidanhoujin Occult Designer Gakuin, Seikimatsu Occult Academy",https://myanimelist.cdn-dena.com/images/anime/5/75257.jpg,TV,Original,13,...,Tuesdays at 01:30 (JST),"{'Adaptation': [{'mal_id': 23365, 'type': 'manga', 'url': 'https://myanimelist.net/manga/23365/S...","Aniplex, Xebec, TV Tokyo Music","NIS America, Inc.",A-1 Pictures,"Sci-Fi, Mystery, Comedy, Supernatural, School","['""Flying Humanoid (フライングヒューマノイド)"" by Shouko Nakagawa (eps 1-12)']","['""Kimi ga Iru Basho (君がいる場所)"" by Ayahi Takagaki (eps 1-13)']",24.0,2010.0
3,36023,8.164312,Persona 5 the Animation,NaN,PERSONA5 the Animation,NaN,https://myanimelist.cdn-dena.com/images/anime/1797/91801.jpg,TV,Game,24,...,Sundays at 00:30 (JST),"{'Adaptation': [{'mal_id': 99944, 'type': 'manga', 'url': 'https://myanimelist.net/manga/99944/P...",CloverWorks,Aniplex of America,A-1 Pictures,"Action, Supernatural, Fantasy","['""BREAK IN TO BREAK OUT"" by Lyn']","['""INFINITY"" by Lyn']",24.0,2018.0
4,26,8.148885,Texhnolyze,Texhnolyze,TEXHNOLYZE,Technolyze,https://myanimelist.cdn-dena.com/images/anime/3/18165.jpg,TV,Original,22,...,Thursdays at 03:28 (JST),[],"Group TAC, Rondo Robe, Fuji TV, Pioneer LDC","Funimation, Geneon Entertainment USA",Madhouse,"Action, Sci-Fi, Psychological, Drama","['""Guardian Angel (Xavier&#039;s Edit)"" by Juno Reactor (eps 1-20, 22)']","['#1: ""Tsuki no Uta (月の詩)"" by Gackt (eps 1-21)', '#2: ""Walking Through the Empty Age"" by Yoko Is...",23.0,2003.0
5,4415,8.143258,World Destruction: Sekai Bokumetsu no Rokunin,Sands of Destruction,ワールド・デストラクション ～世界撲滅の六人～,NaN,https://myanimelist.cdn-dena.com/images/anime/3/30347.jpg,TV,Game,13,...,Unknown,"{'Adaptation': [{'mal_id': 11408, 'type': 'manga', 'url': 'https://myanimelist.net/manga/11408/W...",Sega,Funimation,Production I.G,"Adventure, Fantasy","['""ZERO"" by AAA']","['""Kaze no Kioku～to the end of the world～"" by Aimmy']",25.0,2008.0
6,1210,8.134376,NHK ni Youkoso!,Welcome to the N.H.K.,N・H・Kにようこそ！,Welcome to the NHK,https://myanimelist.cdn-dena.com/images/anime/3/52675.jpg,TV,Novel,24,...,Unknown,"{'Adaptation': [{'mal_id': 463, 'type': 'manga', 'url': 'https://myanimelist.net/manga/463/NHK_n...","NHK, Victor Entertainment","ADV Films, Funimation",Gonzo,"Comedy, Psychological, Drama, Romance","['#1: ""Puzzle"" by ROUND TABLE feat. Nino (eps 1-12)', '#2: ""Puzzle -extra hot mix-"" by ROUND TAB...","['#1: ""Odoru Akachan Ningen"" by Kenji Ohtsuki and Fumihiko Kitsutaka (eps 1-12)', '#2: ""Modokash...",24.0,2006.0
7,1740,8.116013,Naikaku Kenryoku Hanzai Kyousei Torishimarikan Zaizen Joutarou,Government Crime Investigation Agent Zaizen Jotaro,内閣権力犯罪強制取締官・財前丈太郎,Naikaku Kenryoku Hanzai Kyosei Torishimarikan Zaizen Jotaro,https://myanimelist.cdn-dena.com/images/anime/11/62657.jpg,TV,Manga,11,...,Thursdays at 02:40 (JST),"{'Adaptation': 

In [59]:
pd.options.display.max_colwidth = 100

In [63]:
dict(resultDF.loc[0, : ])

{'anime_id': 2382,
 'pred_score': 8.602706053304273,
 'title': 'Firestorm',
 'title_english': nan,
 'title_japanese': 'FIRESTORM\u3000ファイアーストーム',
 'title_synonyms': nan,
 'image_url': 'https://myanimelist.cdn-dena.com/images/anime/11/6090.jpg',
 'type': 'TV',
 'source': 'Original',
 'episodes': 26,
 'status': 'Finished Airing',
 'airing': False,
 'aired_string': 'Apr 6, 2003 to Sep 28, 2003',
 'aired': "{'from': '2003-04-06', 'to': '2003-09-28'}",
 'duration': '25 min. per ep.',
 'rating': 'None',
 'score': 5.83,
 'scored_by': 115,
 'rank': 7726.0,
 'popularity': 8678,
 'members': 805,
 'favorites': 1,
 'background': nan,
 'premiered': 'Spring 2003',
 'broadcast': 'Sundays at 09:00 (JST)',
 'related': '[]',
 'producer': 'Madhouse, TV Tokyo',
 'licensor': 'Enoki Films',
 'studio': 'Trans Arts',
 'genre': 'Action, Military, Sci-Fi, Adventure, Mecha',
 'opening_theme': '[\'"Stormy Love" by the NaB&#039;s\']',
 'ending_theme': '[\'#1: "Kimi to Ita Chikyuu (君といた地球)" by The NaB&#039;s (eps 1

In [58]:
resultDF[resultDF.anime_id == 2382]['image_url']

0    https://myanimelist.cdn-dena.com/images/anime/...
Name: image_url, dtype: object

In [54]:
list(resultDF['title'])

['Firestorm',
 'Otome Youkai Zakuro',
 'Seikimatsu Occult Gakuin',
 'Persona 5 the Animation',
 'Texhnolyze',
 'World Destruction: Sekai Bokumetsu no Rokunin',
 'NHK ni Youkoso!',
 'Naikaku Kenryoku Hanzai Kyousei Torishimarikan Zaizen Joutarou',
 'Ginga Tetsudou Monogatari',
 'Genshiken',
 'Captain Tsubasa (2018)',
 'Kaze no Matasaburou (Movie)',
 'Ookami-san to Shichinin no Nakama-tachi',
 'Monster Hunter Stories: Ride On',
 'Isekai no Seikishi Monogatari',
 'Naruto: Shippuuden Movie 4 - The Lost Tower',
 'Oshiete! Galko-chan: Natsuyasumitte Hontou Desu ka?',
 'Tetsuwan Birdy Decode',
 'Final Fantasy VII: Advent Children Complete',
 'Nijiiro Hotaru: Eien no Natsuyasumi',
 'School Days',
 'Granblue Fantasy The Animation',
 'Koihime†Musou',
 'Ookami Shoujo to Kuro Ouji',
 'Noramimi',
 'Sinbad: Soratobu Hime to Himitsu no Shima',
 'I&#039;&#039;s',
 'Mouretsu Pirates',
 'Major S1',
 'Overlord Movie 1: Fushisha no Ou',
 'Senki Zesshou Symphogear',
 'Colorful Ninja Iromaki',
 'Claymore',


In [38]:
resultDF.to_csv('../../../Downloads/myanimelist/resultDF.csv', index=False)

In [42]:
resultDF.columns

Index(['anime_id', 'pred_score', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme', 'duration_min',
       'aired_from_year'],
      dtype='object')

In [51]:
resultDF[['title', 'rating', 'score', 'scored_by', 'background', 'duration']]

,title,rating,score,scored_by,background,duration
0,Firestorm,None,5.83,115,NaN,25 min. per ep.
1,Otome Youkai Zakuro,PG-13 - Teens 13 or older,7.56,33699,NaN,24 min. per ep.
2,Seikimatsu Occult Gakuin,R - 17+ (violence & profanity),7.12,33630,Alongside So Ra No Wo To and Senkou no Night R...,24 min. per ep.
3,Persona 5 the Animation,R - 17+ (violence & profanity),7.34,11332,NaN,24 min. per ep.
4,Texhnolyze,R+ - Mild Nudity,7.78,29726,NaN,23 min. per ep.
5,World Destruction: Sekai Bokumetsu no Rokunin,PG-13 - Teens 13 or older,7.06,18725,NaN,25 min. per ep.
6,NHK ni Youkoso!,R+ - Mild Nudity,8.38,189128,NaN,24 min. per ep.
7,Naikaku Kenryoku Hanzai Kyousei Torishimarikan...,R - 17+ (violence & profanity),5.78,124,NaN,24 min. per ep.
8,Ginga Tetsudou Monogatari,PG-13 - Teens 13 or older,7.27,1882,NaN,24 min. per ep.
9,Genshiken,PG-13 - Teens 13 or older,7.72,53796,NaN,25 min. per ep.


In [64]:
import requests
import bs4

In [75]:
clear_HTML_re = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def clear_HTML(text):
    return re.sub(clear_HTML_re, '', text)

In [66]:
url = 'https://myanimelist.net/anime/' + '1' + '/'
brs_HTML = requests.get(url)
brs_Soup = bs4.BeautifulSoup(brs_HTML.text)

In [81]:
synopsis = brs_Soup.find('span', itemprop="description")
print(clear_HTML(str(synopsis)))

In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals.

Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member's dark and mysterious past little by little.

Well-balanced with high density action and light-hearted comedy, Cowboy Bebop is a space Western classic an

In [82]:
image = brs_Soup.find('img', itemprop="image")['data-src']
image

'https://cdn.myanimelist.net/images/anime/4/19644.jpg'